Refactor function for taking in array of tickers --> dataframe with correlation values for each ticker
Temporal shift for correlation values
Model features? 

In [62]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import yfinance as yf
from pandas.api.types import is_numeric_dtype

In [10]:
#Pull significant events data
sig_events_df = pd.read_csv('../Data/GDELT_data/article_counts_disasters.csv')
#Drop duplicate index column
sig_events_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
sig_events_df

datetime  All Articles  MANMADE_DISASTER_IMPLIED  \
0     2017-01-01 00:00:00+00:00        427906                     87559   
1     2017-01-02 00:00:00+00:00        592774                    109720   
2     2017-01-03 00:00:00+00:00        757916                    129371   
3     2017-01-04 00:00:00+00:00        822862                    134633   
4     2017-01-05 00:00:00+00:00        830028                    143110   
...                         ...           ...                       ...   
2132  2022-11-06 00:00:00+00:00        166754                     30718   
2133  2022-11-07 00:00:00+00:00        281263                     43567   
2134  2022-11-08 00:00:00+00:00        306702                     44237   
2135  2022-11-09 00:00:00+00:00        327495                     45854   
2136  2022-11-10 00:00:00+00:00        288809                     43763   

      WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE  \
0                                                  3363             
1                                                  4412             
2                                                  5765             
3                                                  6477             
4                                                  6919             
...                                                 ...             
2132                                               8280             
2133                                              14414             
2134                                              15191             
2135                                              18228             
2136                                              16023             

      DISASTER_FIRE  NATURAL_DISASTER_ICE  NATURAL_DISASTER_EARTHQUAKE  \
0             11765                  4972                         1956   
1             11562                  6742                         3079   
2             14952                  8754                         3451   
3             12470                  9553                         5877   
4             11297                 11397                         3690   
...             ...                   ...                          ...   
2132           2963                  1681                          603   
2133           3791                  2577                          663   
2134           3234                  2124                          754   
2135           3363                  2210                         2120   
2136           3356                  2209                          933   

      NATURAL_DISASTER_FLOOD  NATURAL_DISASTER_FLOODING  \
0                       1689                       1309   
1                       1863                       1787   
2                       3366                       3629   
3                       4216                       2672   
4                       5559                       4315   
...                      ...                        ...   
2132                    1362                       1310   
2133                    1659                       2169   
2134                    1771                       2083   
2135                    1760                       2171   
2136                    1517                       2373   

      NATURAL_DISASTER_HURRICANE  ...  NATURAL_DISASTER_FRIGID  \
0                            459  ...                      384   
1                            589  ...                      259   
2                           1222  ...                      385   
3                           2723  ...                      447   
4                           1679  ...                      650   
...                          ...  ...                      ...   
2132                         626  ...                       30   
2133                        1373  ...                       31   
2134                        1924  ...                       40   
2135                        2629  .

In [11]:
#Pull stock data
BP = yf.Ticker("BP")
XEL = yf.Ticker("XEL")
SHEL = yf.Ticker("SHEL")

start = "2017-01-01"
end = "2022-11-10"
interval = '1d'
dataBP = BP.history(period="max",
    interval = interval,
    start = start,
    end = end
)
dataXEL = XEL.history(period="max",
    interval = interval,
    start = start,
    end = end
)
dataSHEL = SHEL.history(period="max",
    interval = interval,
    start = start,
    end = end
)

In [16]:
print(len(dataBP), len(dataXEL), len(dataSHEL))

1475 1475 1475


In [14]:
dataBP.head()

Open       High        Low      Close  \
Date                                                                    
2017-01-03 00:00:00-05:00  26.769708  26.783761  26.551899  26.699448   
2017-01-04 00:00:00-05:00  26.734575  26.938334  26.657286  26.903204   
2017-01-05 00:00:00-05:00  26.797808  27.177222  26.797808  27.099934   
2017-01-06 00:00:00-05:00  26.811856  26.832934  26.594044  26.636202   
2017-01-09 00:00:00-05:00  26.404343  26.453526  26.214636  26.214636   

                            Volume  Dividends  Stock Splits  
Date                                                         
2017-01-03 00:00:00-05:00  8779200        0.0             0  
2017-01-04 00:00:00-05:00  6883300        0.0             0  
2017-01-05 00:00:00-05:00  6505700        0.0             0  
2017-01-06 00:00:00-05:00  5800900        0.0             0  
2017-01-09 00:00:00-05:00  5533600        0.0             0

In [38]:
#Combine all closing prices together
closing_df = pd.DataFrame(list(zip(dataBP.index, dataBP['Close'], 
                                   dataXEL['Close'], dataSHEL['Close'])), columns = ['Date','BP', 'XCEL', 'SHELL'])
closing_df.head()

Date         BP       XCEL      SHELL
0 2017-01-03 00:00:00-05:00  26.699448  34.428505  40.906982
1 2017-01-04 00:00:00-05:00  26.903204  34.581078  41.321823
2 2017-01-05 00:00:00-05:00  27.099934  34.581078  41.684826
3 2017-01-06 00:00:00-05:00  26.636202  34.682789  41.329231
4 2017-01-09 00:00:00-05:00  26.214636  34.157295  40.447689

In [43]:
#Create combined dataframe for stock data
stock_df = pd.DataFrame(columns = ['Date'])
#Create full list of dates from start to end, with a day frequency. Timezone is so it matches timezone of stock data and
#can be merged with no issues
stock_df['Date'] = pd.date_range(start='1/1/2017', end='11/10/2022', freq = 'D', tz = 'America/New_York')
stock_df = stock_df.merge(closing_df, how='outer', on='Date')
stock_df.head(10)

Date         BP       XCEL      SHELL
0 2017-01-01 00:00:00-05:00        NaN        NaN        NaN
1 2017-01-02 00:00:00-05:00        NaN        NaN        NaN
2 2017-01-03 00:00:00-05:00  26.699448  34.428505  40.906982
3 2017-01-04 00:00:00-05:00  26.903204  34.581078  41.321823
4 2017-01-05 00:00:00-05:00  27.099934  34.581078  41.684826
5 2017-01-06 00:00:00-05:00  26.636202  34.682789  41.329231
6 2017-01-07 00:00:00-05:00        NaN        NaN        NaN
7 2017-01-08 00:00:00-05:00        NaN        NaN        NaN
8 2017-01-09 00:00:00-05:00  26.214636  34.157295  40.447689
9 2017-01-10 00:00:00-05:00  26.074116  34.148815  40.277298

In [45]:
stock_df.ffill(axis=0, inplace = True)
stock_df.head(10)

Date         BP       XCEL      SHELL
0 2017-01-01 00:00:00-05:00        NaN        NaN        NaN
1 2017-01-02 00:00:00-05:00        NaN        NaN        NaN
2 2017-01-03 00:00:00-05:00  26.699448  34.428505  40.906982
3 2017-01-04 00:00:00-05:00  26.903204  34.581078  41.321823
4 2017-01-05 00:00:00-05:00  27.099934  34.581078  41.684826
5 2017-01-06 00:00:00-05:00  26.636202  34.682789  41.329231
6 2017-01-07 00:00:00-05:00  26.636202  34.682789  41.329231
7 2017-01-08 00:00:00-05:00  26.636202  34.682789  41.329231
8 2017-01-09 00:00:00-05:00  26.214636  34.157295  40.447689
9 2017-01-10 00:00:00-05:00  26.074116  34.148815  40.277298

In [48]:
#Get correlation to significant events per stock ticker (pearson)
BP_corr_pearson = sig_events_df.corrwith(stock_df.BP)
XCEL_corr_pearson = sig_events_df.corrwith(stock_df.XCEL)
SHELL_corr_pearson = sig_events_df.corrwith(stock_df.SHELL)

In [64]:
corr_pearson_df = pd.DataFrame(list(zip(BP_corr_pearson.index,BP_corr_pearson, XCEL_corr_pearson, SHELL_corr_pearson)),
                               columns = ['GDELT_theme','BP_corr', 'XCEL_corr', 'SHELL_corr'])
corr_pearson_df.set_index('GDELT_theme', inplace = True)
corr_pearson_df

BP_corr  XCEL_corr  \
GDELT_theme                                                               
All Articles                                        0.225351  -0.723220   
MANMADE_DISASTER_IMPLIED                            0.274029  -0.786929   
WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISAS... -0.679025   0.460685   
DISASTER_FIRE                                       0.334232  -0.629724   
NATURAL_DISASTER_ICE                                0.262791  -0.667559   
...                                                      ...        ...   
SELF_IDENTIFIED_ENVIRON_DISASTER                    0.104316  -0.145705   
NATURAL_DISASTER_ERUPTIONS                          0.230594  -0.228077   
NATURAL_DISASTER_TWISTER                            0.100971  -0.226048   
NATURAL_DISASTER_BUSHFIRE                           0.088169  -0.048921   
NATURAL_DISASTER_FLASH_FLOOD                        0.132666  -0.183582   

                                                    SHELL_corr  
GDELT_theme                                                     
All Articles                                          0.147654  
MANMADE_DISASTER_IMPLIED                              0.193023  
WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISAS...   -0.696289  
DISASTER_FIRE                                         0.269574  
NATURAL_DISASTER_ICE                                  0.217866  
...                                                        ...  
SELF_IDENTIFIED_ENVIRON_DISASTER                      0.066750  
NATURAL_DISASTER_ERUPTIONS                            0.205281  
NATURAL_DISASTER_TWISTER                              0.073605  
NATURAL_DISASTER_BUSHFIRE                             0.053810  
NATURAL_DISASTER_FLASH_FLOOD                          0.106319  

[93 rows x 3 columns]

In [70]:
print(corr_pearson_df['BP_corr'].nlargest(5))
print(corr_pearson_df['BP_corr'].nsmallest(5))

GDELT_theme
MANMADE_DISASTER_ROAD_ACCIDENT    0.340741
DISASTER_FIRE                     0.334232
NATURAL_DISASTER_DROWN            0.281246
MANMADE_DISASTER_IMPLIED          0.274029
MANMADE_DISASTER_CAR_ACCIDENT     0.273006
Name: BP_corr, dtype: float64
GDELT_theme
WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE   -0.679025
WB_1705_DISASTER_PREPAREDNESS                                 -0.118539
WB_820_DISASTER_RISK_MANAGEMENT                               -0.074141
NATURAL_DISASTER_TROPICAL_STORM                               -0.008457
NATURAL_DISASTER_WINTER_STORM                                  0.019372
Name: BP_corr, dtype: float64


In [71]:
print(corr_pearson_df['XCEL_corr'].nlargest(5))
print(corr_pearson_df['XCEL_corr'].nsmallest(5))

GDELT_theme
WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE    0.460685
NATURAL_DISASTER_EXTREME_WEATHER                               0.039346
NATURAL_DISASTER_HEAT_WAVE                                     0.018274
NATURAL_DISASTER_HEATWAVE                                      0.002933
NATURAL_DISASTER_DROUGHTS                                     -0.000418
Name: XCEL_corr, dtype: float64
GDELT_theme
MANMADE_DISASTER_ROAD_ACCIDENT      -0.853846
MANMADE_DISASTER_IMPLIED            -0.786929
MANMADE_DISASTER_TRAFFIC_ACCIDENT   -0.775895
All Articles                        -0.723220
NATURAL_DISASTER_ICE                -0.667559
Name: XCEL_corr, dtype: float64


In [72]:
print(corr_pearson_df['SHELL_corr'].nlargest(5))
print(corr_pearson_df['SHELL_corr'].nsmallest(5))

GDELT_theme
MANMADE_DISASTER_ROAD_ACCIDENT    0.295317
DISASTER_FIRE                     0.269574
NATURAL_DISASTER_DROUGHT          0.244546
NATURAL_DISASTER_DROWN            0.226630
MANMADE_DISASTER_CAR_ACCIDENT     0.222706
Name: SHELL_corr, dtype: float64
GDELT_theme
WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE   -0.696289
WB_1705_DISASTER_PREPAREDNESS                                 -0.179899
WB_820_DISASTER_RISK_MANAGEMENT                               -0.144879
NATURAL_DISASTER_TROPICAL_STORM                               -0.023991
MANMADE_DISASTER_OIL_SPILL                                    -0.002927
Name: SHELL_corr, dtype: float64
